In [4]:
from google.colab import files
uploaded = files.upload()

Saving helper.py to helper.py
Saving project_tests.py to project_tests.py


In [5]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.5MB/s 


In [6]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.9.0
Default GPU Device: /device:GPU:0


In [7]:
#https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    
    vgg_tag = 'vgg16'
     
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    
    return w1, keep, layer3, layer4, layer7
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [10]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # Doing Skip Layers
    vgg_layer7_conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same', 
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    vgg_layer4_conv_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same', 
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    vgg_layer3_conv_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same', 
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    
    #deconv layer,upsampling and skip
    #kernel_size=4, strides=2
    layer7_output = tf.layers.conv2d_transpose(vgg_layer7_conv_1x1, num_classes, 4, 2, padding='same', 
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    
    skip_layer_4 = tf.add(layer7_output, vgg_layer4_conv_1x1)
    
    layer4_output = tf.layers.conv2d_transpose(skip_layer_4, num_classes, 4, 2, padding='same', 
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    
    skip_layer_3 = tf.add(layer4_output, vgg_layer3_conv_1x1)
    
    output_layer = tf.layers.conv2d_transpose(skip_layer_3, num_classes, 16, 8, padding='same',  kernel_initializer= tf.random_normal_initializer(stddev=0.01),
                                             kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3));
    
    
    return output_layer
tests.test_layers(layers)

Tests Passed


In [12]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))

    trainOptimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return logits, trainOptimizer, cross_entropy_loss
tests.test_optimize(optimize)


Tests Passed


In [0]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    for epoch in range(epochs):
      for image, lable in get_batches_fn(batch_size):
        _, loss = sess.run([train_op, cross_entropy_loss],
            feed_dict={input_image: image, correct_label: lable, keep_prob:0.75, learning_rate:0.0001})
        print("Loss: = {:.2f}".format(loss))
        
tests.test_train_nn(train_nn)

In [0]:
!wget  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip

In [0]:
!unzip data_road.zip

In [33]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = ''
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        
        layer_output = layers(layer3_out, layer4_out, layer7_out, num_classes)
                              
        correct_label = tf.placeholder(tf.int32, shape = [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32)
                              
        logits, train_op, cross_entropy_loss = optimize(layer_output, correct_label, learning_rate, num_classes)
        sess.run(tf.global_variables_initializer())
        
        epochs = 30
        batch_size = 5

        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image, correct_label, keep_prob, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()


INFO:tensorflow:Restoring parameters from vgg/variables/variables
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [0]:
!zip -r sdcsruns.zip runs/

In [0]:
files.download('sdcsruns.zip')

In [23]:
run()

ERROR:root:File `'().py'` not found.
